In [1]:
import cv2
print(cv2.__version__)
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier

4.1.1


In [3]:
img_full = cv2.imread('full_image.jpg')
img_bkg = cv2.imread('background_image.jpg')
img_pallet = cv2.imread('pallet_image.jpg')

In [2]:
def show(img):
 cv2.imshow('image',img)
 cv2.waitKey(0)
 cv2.destroyAllWindows()

In [4]:
def bgr_to_cmyk(bgr):
    RGB_SCALE = 255
    CMYK_SCALE = 100
    b = bgr[0]
    g = bgr[1]
    r = bgr[2]
    if (r, g, b) == (0, 0, 0):
        return 0, 0, 0, CMYK_SCALE

    c = 1 - r / RGB_SCALE
    m = 1 - g / RGB_SCALE
    y = 1 - b / RGB_SCALE

    min_cmy = min(c, m, y)
    c = (c - min_cmy) / (1 - min_cmy)
    m = (m - min_cmy) / (1 - min_cmy)
    y = (y - min_cmy) / (1 - min_cmy)
    k = min_cmy

    return [c, m, y, k]

def flatten(img):
    return np.reshape(img, (img.shape[0]*img.shape[1],img.shape[2]))

def features(bgr, channels, filter_white=True):
    hsv = cv2.cvtColor(bgr, cv2.COLOR_BGR2HSV)
    cmyk = np.apply_along_axis(bgr_to_cmyk, 2, bgr)
    lab = cv2.cvtColor(bgr, cv2.COLOR_BGR2LAB)
    
    hsv_flat = flatten(hsv)
    lab_flat = flatten(lab)
    cmyk_flat = flatten(cmyk)
    bgr_flat = flatten(bgr)
    
    f = np.concatenate((bgr_flat, hsv_flat, cmyk_flat, lab_flat),1)
    f = [x for x in f if not np.sum(x[:3] == [255,255,255]) == 3] #remove white pixels
    f = np.array(f)
    channels_map = {'B':0, 'G':1,'R':2,'H':3,'S':4,'V':5,'C':6,'M':7,'Y':8,'K':9,'L':10,'A':11,'b':12}
    channels = list(channels)
    channels = [channels_map[ch] for ch in channels]
    f = f[:,channels]
    return np.array(f)

## Select features(channels)

In [ ]:
channels = "HSCMYb"

In [30]:
img_full_features = features(img_full, channels)
img_bkg_features = features(img_bkg, channels)
img_pallet_features = features(img_pallet, channels)

In [31]:
X = np.concatenate((img_bkg_features, img_pallet_features))
y = np.concatenate(([0]*len(img_bkg_features), [1]*len(img_pallet_features)))

In [32]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
sss.get_n_splits(X, y)
      
train_index, test_index = sss.split(X, y).__next__()
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]

## Gaussian Naive Bayes

In [33]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred=gnb.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[51265,  3736],
       [ 1250,  4061]])

## Random Forest

In [35]:
clf = RandomForestClassifier(n_estimators=300, max_depth=10, random_state=0)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
print(clf.feature_importances_)
confusion_matrix(y_test, y_pred)

[0.20471821 0.09534628 0.09294752 0.05058206 0.36727653 0.1891294 ]


array([[53624,  1377],
       [ 1890,  3421]])

In [34]:
def detect(img, model):
    w,h,ch = img.shape
    f = features(img, channels, False)
    pred = model.predict(f)
    img_pred = np.reshape(pred, (w,h,1))
    return np.logical_not(img_pred)

In [36]:
img_classified = detect(img_full, gnb).astype('float32') 

## Intensify result

In [38]:
kernel = np.ones((5,5),np.float32)/25
dst = cv2.filter2D(img_classified,-1,kernel)
dst = cv2.threshold(dst,0.5,1.0,cv2.THRESH_BINARY)[1]
show(dst)

In [12]:
#todo kolejny channel
#todo cecha to średnia z kilku sąsiadów

In [13]:
#Results for channels combinations (Naive Bayes)
#all
array([[53592,  1409],
       [ 3527,  1784]]) #5311
#HSCY
array([[52410,  2591],
       [ 1819,  3492]])
#HSCMY
array([[52307,  2694],
       [ 1742,  3569]])
#HSMY
array([[52845,  2156],
       [ 2697,  2614]])
#HCMY
array([[53081,  1920],
       [ 3070,  2241]])
#SCMY
array([[51814,  3187],
       [ 2338,  2973]])
#HSCMYb
array([[51265,  3736],
       [ 1250,  4061]])
#RGBVK psują

NameError: name 'array' is not defined